In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
path = "../tests/prompts.txt"
with open(path, "r") as f:
    texts = f.readlines()
    print(f"Loaded {len(texts)} prompts from {path}")
    

texts = texts[:40] 
    
messeges = [[{"role": "user", "content": tx}] for tx in texts]

In [ ]:
from gserve.vllm_service import VLLMService
from gserve.configs import LLMConfig, ServeConfig

from tqdm.auto import tqdm
from vllm import SamplingParams


llm_config = LLMConfig(
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    dtype="bfloat16",
)

serve_config = ServeConfig(
    gpu_ids=[1],
    host="127.0.0.1",
    port=None,
    startup_timeout=10 * 60,
    client_timeout=60,
    verbose=True,
)

sampling_params = SamplingParams(
    temperature=0.0,
    n=1,
    max_tokens=100,
)

service = VLLMService(llm_config, serve_config)

service.start()

for i in tqdm(range(10)):
    chat_answers = service.chat(messeges, sampling_params)
    gen_answers = service.generate(texts, sampling_params)

service.shutdown()